In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from PIL import Image
import yaml

In [ ]:
test_img = "images/a.jpg" 
yaml_data = "yaml_data/demo.txt"
show_ids = True        
cap = cv2.VideoCapture(test_img)
loaded_model = tf.keras.models.load_model('empty_vs_occupied.h5')

def run_classifier(img, id):
    img = cv2.resize(img, (64, 64))
    img = Image.fromarray(img) # Returns an image object from array
    img = img.convert(mode='RGB') # convert to RGB 3-channel
    img = image.img_to_array(img, data_format = None)  # data uint8 not allowed in the list of allowed values
    img = np.expand_dims(img, axis = 0)  # add one more dimension
    result = loaded_model.predict(img) 
    if result[0][0]==1: # if empty
        return False
    else:
        return True

In [ ]:
# Read parking slot co-ordinates
with open(yaml_data, 'r') as stream:
    parking_data = yaml.load(stream)
    
parking_slots = []
if parking_data != None:
    for park in parking_data:
        points = np.array(park['points'])
        rect = cv2.boundingRect(points)  # points to rectangle
        parking_slots.append(rect)

C:\Users\Lenovo\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def print_parkIDs(park, coor_points, frame):
    moments = cv2.moments(coor_points)
    centroid = (int(moments['m10']/moments['m00']-5), int(moments['m01']/moments['m00']))
    cv2.putText(frame, str(park['id']), centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

In [ ]:
ret, frame_initial = cap.read()
count_green = 0
if ret == True:
    frame = cv2.resize(frame_initial, None, fx=1, fy=0.7)
    frame_blur = cv2.GaussianBlur(frame.copy(), (5,5), 3)
    frame_gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)

# changing the color on the basis on status and putting numbers on slots
for ind, park in enumerate(parking_data):
    points = np.array(park['points'])
    rect = parking_slots[ind]
    roi_gray = frame_gray[rect[1]:(rect[1] + rect[3]), rect[0]:(rect[0] + rect[2])]  # crop roi for faster calcluation
    
    result = run_classifier(roi_gray, ind)
    if result == False:
        color = (0,255,0)
        count_green = count_green + 1
    else:
        color = (0,0,255)    
            
    cv2.drawContours(frame, [points], contourIdx=-1, color=color, thickness=3, lineType=cv2.LINE_8) # draw rectangle
    if show_ids:
        print_parkIDs(park, points, frame)
            
cv2.putText(frame, 'No of available spots: ' +str(count_green), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
cv2.imshow('frame', frame)
cv2.waitKey(0)   # displays image until keypress 
cap.release()
cv2.destroyAllWindows()